FINE-TUNING PHRASE

In [ ]:
# Installing libraries
!pip install transformers
!pip install torch
!pip install sentencepiece
!pip install datasets
# !pip install sacrebleu
!pip install transformers[torch] -U
!pip install accelerate -U
!pip install evaluate

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import numpy as np
import evaluate

In [ ]:
# Loading the dataset
# dataset = load_dataset("csv", data_files="D:/Study Documents/University of Edinburgh/MSc/Dissertation/data/cut/part1.csv")

# Loading dataset to colab
from google.colab import files

uploaded = files.upload()

Saving date_rows_tab_trans.csv to date_rows_tab_trans (4).csv


In [ ]:
# Inspecting the dataset to check if each row has 3 fields only (especially line 1365)

lines = []
with open('date_rows_tab_trans.csv', 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        if 1360 <= i <= 1370:
            lines.append(line)

lines

['DATE\t22 April 2013\tthe twenty second of april twenty thirteen\n',
 'DATE\t2000\ttwo thousand\n',
 'DATE\tFebruary 11 1982\tfebruary eleventh nineteen eighty two\n',
 'DATE\t4 August 2006\tthe fourth of august two thousand six\n',
 'DATE\tThursday\t October 22\t 2009\tthursday october twenty second two thousand nine\n',
 'DATE\t6 June 1989\tthe sixth of june nineteen eighty nine\n',
 'DATE\t2004\ttwo thousand four\n',
 'DATE\t14 July 1905\tthe fourteenth of july nineteen o five\n',
 'DATE\t2000\ttwo thousand\n',
 'DATE\tDecember 17 2009\tdecember seventeenth two thousand nine\n',
 'DATE\t1806\teighteen o six\n']

In [ ]:
# Standardizing the number of fields
from google.colab import files

# Upload the file
uploaded = files.upload()

# List of uploaded files (since we expect only one, we take the first one)
input_file = list(uploaded.keys())[0]
output_file = 'colab_dates.csv'

def transform_csv(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile, \
            open(output_file, 'w', encoding='utf-8', newline='') as outfile:

        for line in infile:
            # Split the line by tabs and filter out empty fields
            fields = [field for field in line.strip().split('\t') if field]

            if len(fields) == 5:
                # Combine fields 2, 3, and 4
                fields[1] = fields[1] + fields[2] + fields[3]
                del fields[2:4]  # Remove the original third and fourth fields

            # Write the modified line to the output file
            outfile.write('\t'.join(fields) + '\n')

# Call the function to transform the CSV
transform_csv(input_file, output_file)

# Download the output file
files.download(output_file)

Saving date_rows_tab_trans.csv to date_rows_tab_trans (3).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Loading the dataset (modified for colab)
from google.colab import files

uploaded = files.upload()

Saving colab_dates.csv to colab_dates (2).csv


In [ ]:
# Reading the file
# df = pd.read_csv('cut_1.csv', encoding='cp1252', sep=',')  # included sep "," here because delimiter examination returns this (printing below gives "\t"?)
df = pd.read_csv('colab_dates.csv', encoding="utf-8", delimiter='\t')

In [ ]:
# Inspecting the full dataset
print(df.head(n=50))

   class              input                                         output
0   DATE               2007                             two thousand seven
1   DATE               2008                             two thousand eight
2   DATE       4 March 2014            the fourth of march twenty fourteen
3   DATE      April 10 2013                    april tenth twenty thirteen
4   DATE      7 August 2007       the seventh of august two thousand seven
5   DATE               1987                          nineteen eighty seven
6   DATE               1996                            nineteen ninety six
7   DATE  22 September 2015  the twenty second of september twenty fifteen
8   DATE    November 4 2014                november fourth twenty fourteen
9   DATE               1916                               nineteen sixteen
10  DATE               1982                            nineteen eighty two
11  DATE    7 December 2015         the seventh of december twenty fifteen
12  DATE           11/17/

In [ ]:
# Splitting into 3 columns: class, text, normalization

# Assign names to each field
df.columns = ['class', 'input', 'output']

# Save the modified DataFrame back to a CSV file
df.to_csv('colab_dates.csv', sep='\t', index=False)

In [ ]:
# Inspecting the full dataset (with name of each field)
print(df.head(n=15))

   class              input                                         output
0   DATE               2007                             two thousand seven
1   DATE               2008                             two thousand eight
2   DATE       4 March 2014            the fourth of march twenty fourteen
3   DATE      April 10 2013                    april tenth twenty thirteen
4   DATE      7 August 2007       the seventh of august two thousand seven
5   DATE               1987                          nineteen eighty seven
6   DATE               1996                            nineteen ninety six
7   DATE  22 September 2015  the twenty second of september twenty fifteen
8   DATE    November 4 2014                november fourth twenty fourteen
9   DATE               1916                               nineteen sixteen
10  DATE               1982                            nineteen eighty two
11  DATE    7 December 2015         the seventh of december twenty fifteen
12  DATE           11/17/

In [ ]:
# Creating the Hugging Face dataset using 'input' and 'output' columns
dataset = Dataset.from_pandas(df[['input', 'output']])

In [ ]:
# Splitting into train, valid, test datasets
from sklearn.model_selection import train_test_split

# Step 1: Split the dataset into 90% training + validation and 10% test
train_val_dataset, test_dataset = dataset.train_test_split(test_size=0.1).values()

# Step 2: Split the 90% training + validation dataset into 80% training and 10% validation
train_dataset, val_dataset = train_val_dataset.train_test_split(test_size=0.1111).values()

In [ ]:
# Getting the sizes of the datasets
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

Training set size: 274708
Validation set size: 34335
Test set size: 34339


In [ ]:
# Initializing the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-small')
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-small')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Preprocessing the data function
def preprocess_data(examples):
    inputs = examples['input']
    targets = examples['output']

    # Tokenize inputs
    inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    # Tokenize targets (labels)
    labels = tokenizer(targets, max_length=512, truncation=True, padding='max_length')

    # Adjust labels for T5 model
    labels["input_ids"] = [[label_id if label_id != tokenizer.pad_token_id else -100 for label_id in label_ids] for label_ids in labels["input_ids"]]

    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": labels["input_ids"],
    }

In [ ]:
# Tokenizing the whole dataset
train_dataset = train_dataset.map(preprocess_data, batched=True)
val_dataset = val_dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/274708 [00:00<?, ? examples/s]

Map:   0%|          | 0/34335 [00:00<?, ? examples/s]

Map:   0%|          | 0/34339 [00:00<?, ? examples/s]

In [ ]:
# Setting the format for PyTorch
train_dataset.set_format(type='pt', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='pt', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='pt', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
# Initialize the metric
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(pred):
    # Check if predictions are in tuple format, convert them to tensor if so
    if isinstance(pred.predictions, tuple):
        preds = pred.predictions[0].argmax(axis=-1)
    else:
        preds = pred.predictions.argmax(axis=-1)

    labels = pred.label_ids
    mask = labels != -100
    labels = labels[mask]
    preds = preds[mask]
    return accuracy_metric.compute(predictions=preds, references=labels)

In [ ]:
# Initialize the model
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

# Define training arguments (no batch size for now)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train the model
trainer.train()

# Evaluate the model on the test set
eval_result = trainer.evaluate(eval_dataset=test_dataset)
print(f"Test set evaluation result: {eval_result}")

# Save the final model and tokenizer
trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")

print("Training complete and model saved.")

Epoch,Training Loss,Validation Loss


RUNNING INFERENCE

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

def normalize_text(model, tokenizer, text, max_length=128):
    # Tokenize input text using T5 tokenizer
    input_ids = tokenizer(text, return_tensors="pt").input_ids

    # Move input tensors to the same device as the model
    device = next(model.parameters()).device
    input_ids = input_ids.to(device)

    # Ensure model is in evaluation mode
    model.eval()

    # Generate outputs using the model
    with torch.no_grad():
        outputs = model.generate(input_ids, max_length=max_length)

    # Decode the output tokens using T5 tokenizer
    normalized_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return normalized_text

# Load the tokenizer and model
model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Example inference with increased max_length
text = "30 Dec"
normalized_text = normalize_text(model, tokenizer, text, max_length=128)
print(f"Normalized Text: {normalized_text}")

# Debugging: Print tokenized input and output ids
input_ids = tokenizer(text, return_tensors="pt").input_ids
print(f"Tokenized Input IDs: {input_ids}")

with torch.no_grad():
    outputs = model.generate(input_ids.to(next(model.parameters()).device), max_length=128)
output_ids = outputs[0]
print(f"Output IDs: {output_ids}")

# Decode the output IDs
decoded_output = tokenizer.decode(output_ids, skip_special_tokens=False)
print(f"Decoded Output: {decoded_output}")